# Simulación de Taxis de NYC por el método Montecarlo

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta

### Funciones

In [9]:
def rango(df,column:str):
    '''
Esta función devuelve un rango de valores que excluye outliers, la función recibe
    - df - Un dataFrame
    - column - Una columna en str
    La función devuelve una tupla con el intervalo valido
    '''
    statistics = df[column].describe()
    IQR = statistics['75%'] - statistics['25%']
    RI = statistics['25%'] - 1.5 * IQR       # RAngo inferior
    if RI < 0:
        RI = 0
    RS = statistics['75%'] + 1.5 * IQR       # Rango superior
    if RI == RS:
        return (RI-1,RS)
    else:
        return (RI , RS)

In [65]:
def trip(UpLocationId:int,Hour):
    '''
    Esta función devuelve caracteristicas de la ruta entre dos puntos. La función recibe:
        - UpLocationId. Id de la zona de inicio del viaje
        - Hour. Hora en la que inicia el viaje en formato `timedelta`
        
    La función devuelve un diccionario con:
        - route. Ruta
        - UpLocationId. Id zona de inicio del viaje
        - DOLocationID. Id zona final del viaje
        - Hour_i. Hora de inicio del viaje
        - Hour_e. Hora en la que finaliza el viaje
        - trip_time. Tiempo del viaje en minutos
        - trip_distance. Distancia recorrida
        - total_amount. Costo del viaje
    '''
    filtro = taxi[taxi['PULocationID'] == UpLocationId]             # Filtramos registros de la zona de inicio del viaje
    destino = np.random.choice(filtro['DOLocationID'])              # Definimos zona de destino del viaje
    filtro = filtro[filtro['DOLocationID'] == destino]              # Actualizamos el filtro
    # Determinar el rango de valores que excluyen outliers
    rango_tiempo = rango(filtro,'time')
    rango_distancia = rango(filtro,'trip_distance')
    # Eliminando registros con datos atipicos
    if len(filtro) != 1:
        filtro = filtro[(filtro['time'] >= rango_tiempo[0]) & (filtro['time'] <= rango_tiempo[1]) & 
                    (filtro['trip_distance'] > rango_distancia[0]) &
                      (filtro['trip_distance'] <= rango_distancia[1])]
        route = np.random.choice(filtro['route'])
    else:
        filtro = filtro.reset_index()
        route = filtro['route'].iloc[0]
    # Extrayendo datos
    
    distance = np.random.choice(filtro['trip_distance'])
    time = filtro[filtro['trip_distance'] == distance]['time'].mean()
    amount = filtro[filtro['trip_distance'] == distance]['total_amount'].mean()
    Hour_e = Hour + timedelta(hours=0, minutes = time)
    return {"route":route,"UpLocationId":UpLocationId,"DOLocationID":destino,"Hour_i":Hour,"Hour_e":Hour_e,
            "trip_time":time,"trip_distance":distance,"total_amount":amount}

In [51]:
def simulacion(n:int):
    iteraciones = []
    for j in range(0,n):
        idzonaInicio = np.random.choice(taxi['PULocationID'])
        data = []
        i = timedelta(hours=5,minutes=0)
        while i <= timedelta(hours=18):
            a = trip(idzonaInicio,i)
            data.append(a)
            idzonaInicio = a['DOLocationID']
            i = a['Hour_e']
        df = pd.DataFrame(data)
        b = {"interacion":j,"trip_time":timedelta(minutes=df['trip_time'].sum()),
        "trip_distance":df['trip_distance'].sum(),"total_amount":df['total_amount'].sum()}
        iteraciones.append(b)
    return pd.DataFrame(iteraciones)

Cargando datos

In [11]:
taxi = pd.read_parquet('../../Data/Data_taxi_sim.parquet')
fuel = pd.read_parquet('../../Data/Eda_fuel_vehicles.parquet')

Transformaciones en el df `taxi`, en donde se crea la columna `time` la cual es un entero en minutos|

In [12]:
taxi['time'] = taxi['tpep_dropoff_datetime'] - taxi['tpep_pickup_datetime']
taxi['time'] = taxi['time'].apply(lambda x: int(x.total_seconds() // 60))
taxi = taxi.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'], axis=1)

### Simulaciones

In [54]:
sim2 = simulacion(5)
print(f"La simulacion con {len(sim2)} corridas ofrece la siguiente información:\nEl recorrido simulado es de {sim2['trip_distance'].mean()} millas \nEl ingreso simulado es ${sim2['total_amount'].mean()} \nPara un total de {sim2['trip_time'].mean()} horas")

La simulacion con 5 corridas ofrece la siguiente información:
El recorrido simulado es de 154.83999999999997 millas 
El ingreso simulado es $1217.1262932970735 
Para un total de 0 days 13:17:33.560412800 horas


In [56]:
sim2 = simulacion(10)
print(f"La simulacion con {len(sim2)} corridas ofrece la siguiente información:\nEl recorrido simulado es de {sim2['trip_distance'].mean()} millas \nEl ingreso simulado es ${sim2['total_amount'].mean()} \nPara un total de {sim2['trip_time'].mean()} horas")

La simulacion con 10 corridas ofrece la siguiente información:
El recorrido simulado es de 159.831 millas 
El ingreso simulado es $1644.548067588664 
Para un total de 0 days 13:08:04.402167200 horas


In [58]:
sim2 = simulacion(20)
print(f"La simulacion con {len(sim2)} corridas ofrece la siguiente información:\nEl recorrido simulado es de {sim2['trip_distance'].mean()} millas \nEl ingreso simulado es ${sim2['total_amount'].mean()} \nPara un total de {sim2['trip_time'].mean()} horas")

La simulacion con 20 corridas ofrece la siguiente información:
El recorrido simulado es de 159.054 millas 
El ingreso simulado es $1459.0793305907066 
Para un total de 0 days 13:20:16.832743250 horas


In [59]:
sim2 = simulacion(40)
print(f"La simulacion con {len(sim2)} corridas ofrece la siguiente información:\nEl recorrido simulado es de {sim2['trip_distance'].mean()} millas \nEl ingreso simulado es ${sim2['total_amount'].mean()} \nPara un total de {sim2['trip_time'].mean()} horas")

La simulacion con 40 corridas ofrece la siguiente información:
El recorrido simulado es de 153.861 millas 
El ingreso simulado es $1467.0883756296664 
Para un total de 0 days 13:12:36.157214725 horas


In [61]:
sim2 = simulacion(80)
print(f"La simulacion con {len(sim2)} corridas ofrece la siguiente información:\nEl recorrido simulado es de {sim2['trip_distance'].mean()} millas \nEl ingreso simulado es ${sim2['total_amount'].mean()} \nPara un total de {sim2['trip_time'].mean()} horas")

La simulacion con 80 corridas ofrece la siguiente información:
El recorrido simulado es de 154.84650000000002 millas 
El ingreso simulado es $1562.5812086645026 
Para un total de 0 days 13:12:49.142777200 horas


In [66]:
sim2 = simulacion(200)
print(f"La simulacion con {len(sim2)} corridas ofrece la siguiente información:\nEl recorrido simulado es de {sim2['trip_distance'].mean()} millas \nEl ingreso simulado es ${sim2['total_amount'].mean()} \nPara un total de {sim2['trip_time'].mean()} horas")

La simulacion con 200 corridas ofrece la siguiente información:
El recorrido simulado es de 155.70495 millas 
El ingreso simulado es $1517.3900221458541 
Para un total de 0 days 13:12:12.375146315 horas


In [ ]:
sim2 = simulacion(500)
print(f"La simulacion con {len(sim2)} corridas ofrece la siguiente información:\nEl recorrido simulado es de {sim2['trip_distance'].mean()} millas \nEl ingreso simulado es ${sim2['total_amount'].mean()} \nPara un total de {sim2['trip_time'].mean()} horas")